# NEWHOPE

In [69]:
class NTT(object):
#    
    def __init__(self, n=128):
        if not any([n == t for t in [32,64,128,256,512,1024,2048]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 1 + 2*n
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
            
        self.F = GF(self.q);
        self.R = PolynomialRing(self.F, name="w")
        
        w = (self.R).gen();
        self.w = w
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; 
            xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ;
            f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ;
            ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ;
            ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ;
                b = s*ff1[i]  
                ff[i] = a + b ; 
                ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)
    
    
    #  Como o vetor1 é resultado de uma multiplicação 
    # será sempre de tamanho maior ou igual ao vetor2
    def pol_sum(self,vetor1,vetor2):
        l1 = len(vetor1) ; l2 = len(vetor2)
        for i in range(l1):
            if i < l2:
                vetor1[i] += vetor2[i]
        return vetor1
    
    #  Como o vetor2 é resultado de uma multiplicação 
    # será sempre de tamanho maior ou igual ao vetor1
    def pol_sub(self,vetor1,vetor2):
        l1 = len(vetor1) ; l2 = len(vetor2)
        for i in range(l2):
            if i < l1:
                vetor2[i] = vetor1[i] - vetor2[i]
            else:
                vetor2[i] *= -1
        return vetor2
        
        
    def pol_mul(self,vetor1,vetor2):
        f_vector = [0] * (len(vetor1) + len(vetor2) - 1);
        for i in range(len(vetor1)):
            for j in range(len(vetor2)):
                f_vector[i+j] += vetor1[i] * vetor2[j]
        return f_vector
    
    
    
    #Algoritmo Rq -> gera coeficientes em Rq
    def gen(self,seed):                                            #feito?
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        #gera uma lista de bits que epresentam números entre 0 e 255
        f = implemented_SHAKE256(64,_toStr(seed).encode('utf-8'))
        
        #converte a lista de bytes numa lista de coeficientes de tamanha n
        fLinha = [int(coef) for coef in f]
        return Rq(fLinha)
    
    #seedS <- {0,1}^t
    #s  <- gen1(seedS)
    #sk =  seedS
    #seedA <- {0.1}^t
    #a  <- gen1(seedA)
    #seedE <- {0,1}^t
    #E  <- gen2(seedE)
    #b <- a*s + E
    #pk = (seedA,b)
    #return (pk,sk)
    def keyGen(self):
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        
        #Geração das sementes e tamanho 32 e de polinómios de tamanho 64
        seedS = [choice([0,1]) for k in range(32)] 
        s     = self.gen(seedS)                             # vetor de polinómios (k)
        sk    = s                                      
        seedA = [choice([0,1]) for k in range(32)]
        a     = self.gen(seedA)                            # pequena matriz (k*k)?
        seedE = [choice([0,1]) for k in range(32)]
        e     = self.gen(seedE)                            # vetor de polinómios (k)
        
        #Operações sobre polinómios
        multiplicacao = self.pol_mul(a.list(),s.list())
        b             = self.pol_sum(multiplicacao, e.list())          
        pk            = (seedA,Rq(b))
        return (pk,sk)
       
        
        
    #se seedR is Null then seedR <-{0,1}^t
    #(seedA,b) = pk
    #a = gen1(seedA)
    #r = gen1(seedR)
    #u        <- encode(m)
    # E', E'' <- Xalfa(Rq)
    #c1       <- a*r +E'
    #c2       <- b*r + E'' + u
    #return (c1,c2)
    def Encapsulate(self,pk,m = [],seedR = []):
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        
        #Geração de polinómios
        if len(seedR) == 0:
            seedR = [choice([0,1]) for k in range(32)]
            
        (seedA,b) = pk
        a         = self.gen(seedA)                           # A gerado
        r         = self.gen(seedR)                           # R gerado
        #u = encode(m)
        seedE1    = [choice([0,1]) for k in range(32)]
        seedE2    = [choice([0,1]) for k in range(32)]
        ELinha1   = self.gen(seedE1)                          # E' gerado
        ELinha2   = self.gen(seedE2)                          # E'' gerado
        
        #Operações sobre polinómios
        mult1 = self.pol_mul(a.list(),r.list())
        mult2 = self.pol_mul(b.list(),r.list())
        
        c1 = self.pol_sum(mult1,ELinha1.list())               #por fazer
        c2 = self.pol_sum(mult2,ELinha2.list())
        #c2 = b*r + ELinha2 + u                               #por fazer
        return (Rq(c1),Rq(c2))
    
    
    
    # seedS = sk
    # s = gen1(seedS)
    # m <- decode(c2-s*c1)
    def Decapsulate(self,c1,c2,sk):
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        
        #Geração de polinómios
        seedS = sk
        s = self.gen(seedS)
        #m = decode(c2-s*c1)
        
        #Operações sobre polinómios
        mul = self.pol_mul(s.list(),c1.list())
        res = self.pol_sub(c2.list(), mul)
        return Rq(res)

In [70]:
# Teste NTT

T = NTT(2048)
'''
f = T.random_pol(64)

ff = T.ntt(f)
#print(ff)

fff = T.ntt_inv(ff)
#sprint(fff)

print("\nCorreto ? ",f == fff)
'''

print(T.q)
(pk,sk)=T.keyGen()
(c1,c2) = T.Encapsulate(pk)
x = T.Decapsulate(c1,c2,sk)

12289


In [71]:
print(pk)
print(sk)

([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0], 244*z^126 + 1150*z^125 + 12076*z^124 + 11109*z^123 + 3817*z^122 + 7822*z^121 + 4402*z^120 + 468*z^119 + 1940*z^118 + 6598*z^117 + 11847*z^116 + 10061*z^115 + 10624*z^114 + 9870*z^113 + 10818*z^112 + 10651*z^111 + 1882*z^110 + 11711*z^109 + 7045*z^108 + 6498*z^107 + 9538*z^106 + 955*z^105 + 10793*z^104 + 4728*z^103 + 365*z^102 + 10426*z^101 + 3107*z^100 + 9711*z^99 + 8504*z^98 + 6617*z^97 + 4520*z^96 + 11925*z^95 + 11115*z^94 + 4146*z^93 + 6005*z^92 + 9603*z^91 + 11064*z^90 + 11586*z^89 + 9451*z^88 + 11207*z^87 + 12217*z^86 + 3152*z^85 + 9164*z^84 + 11649*z^83 + 11028*z^82 + 10050*z^81 + 3487*z^80 + 8504*z^79 + 11384*z^78 + 4906*z^77 + 3946*z^76 + 5703*z^75 + 10105*z^74 + 10680*z^73 + 9545*z^72 + 2571*z^71 + 6556*z^70 + 4923*z^69 + 2519*z^68 + 5925*z^67 + 9472*z^66 + 5006*z^65 + 10232*z^64 + 1517*z^63 + 5744*z^62 + 8306*z^61 + 10754*z^60 + 5273*z^59 + 7872*z^58 + 4156*z^57 + 9990*z^56 + 69

In [72]:
print(c1)
print(c2)
print(x)

8064*z^126 + 752*z^125 + 5533*z^124 + 10577*z^123 + 11318*z^122 + 9260*z^121 + 3414*z^120 + 10150*z^119 + 7153*z^118 + 11315*z^117 + 10593*z^116 + 1183*z^115 + 7650*z^114 + 7359*z^113 + 6926*z^112 + 3924*z^111 + 4288*z^110 + 11415*z^109 + 10236*z^108 + 5404*z^107 + 6302*z^106 + 618*z^105 + 6721*z^104 + 12186*z^103 + 6471*z^102 + 9270*z^101 + 10808*z^100 + 1949*z^99 + 10193*z^98 + 10606*z^97 + 11827*z^96 + 4037*z^95 + 2038*z^94 + 6174*z^93 + 2191*z^92 + 2740*z^91 + 9597*z^90 + 172*z^89 + 2296*z^88 + 1064*z^87 + 771*z^86 + 4653*z^85 + 10358*z^84 + 10474*z^83 + 2902*z^82 + 1076*z^81 + 9996*z^80 + 595*z^79 + 1702*z^78 + 9883*z^77 + 4428*z^76 + 8246*z^75 + 3470*z^74 + 6232*z^73 + 1007*z^72 + 4698*z^71 + 11970*z^70 + 3373*z^69 + 10354*z^68 + 1769*z^67 + 3420*z^66 + 5991*z^65 + 10806*z^64 + 5662*z^63 + 578*z^62 + 11823*z^61 + 1932*z^60 + 761*z^59 + 5843*z^58 + 1201*z^57 + 11176*z^56 + 5006*z^55 + 10852*z^54 + 1046*z^53 + 4510*z^52 + 5634*z^51 + 9251*z^50 + 10967*z^49 + 6025*z^48 + 10188*z^47 

In [45]:
import hashlib
import numpy as np
import matplotlib.pyplot as plt

'''
O NewHope usa extensivamente a transformada NTT
e grande parte das operações polinomiais e geração
de polinómios são feitas neste dominio das transformadas 
'''

def implemented_SHAKE256(length, message):
    m = hashlib.shake_256()
    m.update(message)
    return m.digest(int(length))

'''
Usada para converter as seeds em bytes para serem aceites como input
na SKAKE256()
'''
def _toStr(message):
    mystring = ""

    for digit in message:
        mystring += str(digit)
    return mystring


class newHope():
    #n é potência de 2
    # (q = 1 mod 2n) & q é primo
    def __init__(self):
        self.n = 256
        q=2*256
        while True:
            if q.is_prime() and gcd(q,2*self.n):
                break
            else:
                q += 1
       
        self.q = q
        self.k = 8
        self.T = NTT(self.n)
        self.t = 32

        
    def pol_sum(self,vetor1,vetor2):
        for i in range(len(vetor2)):
            vetor1[i] += vetor2[i]
        return vetor1
    
    
    
    def pol_sub(self,vetor1,vetor2):
        for i in range(len(vetor1)):
            vetor1[i] -= vetor2[i]
        return vetor1
        
        
        
    def pol_mul(self,vetor1,vetor2):
        f_vector = []
        for i in range(len(vetor1)*len(vetor2)):
            f_vector.append(0)
        for i in range(len(vetor1)):
            for j in range(len(vetor2)):
                f_vector[i+j] += vetor1[i] * vetor2[j]
        return f_vector

    
    #Algoritmo Rq -> gera coeficientes em Rq
    def gen(self,seed):                                            #feito?
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        #gera uma lista de bits que epresentam números entre 0 e 255
        f = implemented_SHAKE256(64,_toStr(seed).encode('utf-8'))
        
        #converte a lista de bytes numa lista de coeficientes de tamanha n
        fLinha = [int(coef) for coef in f]
        return Rq(fLinha)
     
    
    #seedS <- {0,1}^t
    #s  <- gen1(seedS)
    #sk =  seedS
    #seedA <- {0.1}^t
    #a  <- gen1(seedA)
    #seedE <- {0,1}^t
    #E  <- gen2(seedE)
    #b <- a*s + E
    #pk = (seedA,b)
    #return (pk,sk)
    def keyGen(self):
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        
        seedS = [choice([0,1]) for k in range(32)] 
        s     =self.gen(seedS)
        #sHat  = self.T.ntt(s)                             # vetor de polinómios (k)
        sk    = s                                      
        seedA = [choice([0,1]) for k in range(self.t)]
        a     = self.gen(seedA)                            # pequena matriz (k*k)?
        seedE = [choice([0,1]) for k in range(self.t)]
        e     = self.gen(seedE)
        #eHat  = self.T.ntt(e)                             # vetor de polinómios (k)
        multiplicacao = self.pol_mul(a.list(),s.list())
        b     = self.pol_sum(multiplicacao, e.list())          
        pk    = (seedA,Rq(b))
        return (pk,sk)
       
        
        
    #se seedR is Null then seedR <-{0,1}^t
    #(seedA,b) = pk
    #a = gen1(seedA)
    #r = gen1(seedR)
    #u        <- encode(m)
    # E', E'' <- Xalfa(Rq)
    #c1       <- a*r +E'
    #c2       <- b*r + E'' + u
    #return (c1,c2)
    def Encapsulate(self,pk,m = [],seedR = []):
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        
        if len(seedR) == 0:
            seedR = [choice([0,1]) for k in range(self.t)]
            
        (seedA,b) = pk
        a = self.gen(seedA)
        r = self.gen(seedR)
        #u = encode(m)                                        #por fazer
        seedE1 = [choice([0,1]) for k in range(self.t)]
        seedE2 = [choice([0,1]) for k in range(self.t)]
        ELinha1 = self.som(seedE1)#por fazer
        ELinha2 = self.som(seedE2) 
        
        mult1 = self.pol_mul(a.list(),r.list())
        mult2 = self.pol_mul(b.list(),r.list())
        
        c1 = self.pol_sum(mult1,ELinha1.list())               #por fazer
        c2 = self.pol_sum(mult2,ELinha2.list())
        #c2 = b*r + ELinha2 + u                               #por fazer
        return (Rq(c1),Rq(c2))
    
    
    
    # seedS = sk
    # s = gen1(seedS)
    # m <- decode(c2-s*c1)
    def Decapsulate(self,c1,c2,sk):
        Zq.<z>  = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n-z-1)
        
        seedS = sk
        s = self.gen(seedS)
        #m = decode(c2-s*c1)
        mul = self.pol_mul(s.list(),c1.list())
        res = self.pol_sub(c2.list(), mul)
        return Rq(res)
        

In [46]:
N = newHope()
print(N.q)
(pk,sk)=N.keyGen()
(c1,c2) = N.Encapsulate(pk)
x = N.Decapsulate(c1,c2,sk)

521


In [48]:
print(c1)
print(c2)

367*z^126 + 113*z^125 + 138*z^124 + 254*z^123 + 109*z^122 + 285*z^121 + 232*z^120 + 140*z^119 + 106*z^118 + 373*z^117 + 31*z^116 + 325*z^115 + 426*z^114 + 482*z^113 + 154*z^112 + 272*z^111 + 151*z^110 + 162*z^109 + 387*z^108 + 294*z^107 + 46*z^106 + 416*z^105 + 312*z^104 + 396*z^103 + 10*z^102 + 223*z^101 + 80*z^100 + 249*z^99 + 93*z^98 + 393*z^97 + 35*z^96 + 417*z^95 + 175*z^94 + 283*z^93 + 145*z^92 + 280*z^91 + 369*z^90 + 484*z^89 + 318*z^88 + 238*z^87 + 23*z^86 + 9*z^85 + 374*z^84 + 437*z^83 + 197*z^82 + 192*z^81 + 147*z^80 + 426*z^79 + 289*z^78 + 148*z^77 + 142*z^76 + 419*z^75 + 239*z^74 + 287*z^73 + 303*z^72 + 229*z^71 + 87*z^70 + 385*z^69 + 246*z^68 + 116*z^67 + 314*z^66 + 189*z^65 + 200*z^64 + 420*z^63 + 256*z^62 + 410*z^61 + 257*z^60 + 234*z^59 + 448*z^58 + 260*z^57 + 441*z^56 + 148*z^55 + 516*z^54 + 346*z^53 + 367*z^52 + 122*z^51 + 41*z^50 + 344*z^49 + 430*z^48 + 324*z^47 + 281*z^46 + 105*z^45 + 38*z^44 + 96*z^43 + 162*z^42 + 335*z^41 + 414*z^40 + 153*z^39 + 306*z^38 + 119*z^3

In [50]:
print(c1)
print(c2)
print(x)

515*z^189 + 487*z^188 + 430*z^187 + 363*z^186 + 228*z^185 + 403*z^184 + 87*z^183 + 49*z^182 + 293*z^181 + 245*z^180 + 17*z^179 + 400*z^178 + 75*z^177 + 30*z^176 + 433*z^175 + 40*z^174 + 332*z^173 + 4*z^172 + 197*z^171 + 171*z^170 + 202*z^169 + 55*z^168 + 253*z^167 + 327*z^166 + 313*z^165 + 497*z^164 + 519*z^163 + 448*z^162 + 422*z^161 + 519*z^160 + 168*z^159 + 186*z^158 + 372*z^157 + 195*z^156 + 467*z^155 + 410*z^154 + 138*z^153 + 445*z^152 + 206*z^151 + 301*z^150 + 265*z^149 + 13*z^148 + 450*z^147 + 77*z^146 + 138*z^145 + 349*z^144 + 123*z^143 + 429*z^142 + 455*z^141 + 198*z^140 + 307*z^139 + 270*z^138 + 380*z^137 + 434*z^136 + 156*z^135 + 59*z^134 + 497*z^133 + 266*z^132 + 21*z^131 + 185*z^130 + 280*z^129 + 115*z^128 + 202*z^127 + 310*z^126 + 22*z^125 + 389*z^124 + 127*z^123 + 9*z^122 + 125*z^121 + 244*z^120 + 313*z^119 + 167*z^118 + 86*z^117 + 323*z^116 + 31*z^115 + 151*z^114 + 492*z^113 + 317*z^112 + 131*z^111 + 68*z^110 + 87*z^109 + 393*z^108 + 392*z^107 + 20*z^106 + 435*z^105 + 3